In [6]:
import tkinter as tk
from tkinter import messagebox
import sqlite3

# Function to add task to the database
def add_task():
    task = entry_task.get()
    if task:
        conn.execute("INSERT INTO tasks (task) VALUES (?)", (task,))
        conn.commit()
        update_tasks()
        entry_task.delete(0, tk.END)
    else:
        messagebox.showwarning("Warning", "Please enter a task")

# Function to delete task from the database
def delete_task():
    selection = listbox_tasks.curselection()
    if selection:
        index = selection[0]
        task_id = task_ids[index]
        conn.execute("DELETE FROM tasks WHERE id=?", (task_id,))
        conn.commit()
        update_tasks()
    else:
        messagebox.showwarning("Warning", "Please select a task to delete")

# Function to update the tasks list
def update_tasks():
    listbox_tasks.delete(0, tk.END)
    for index, row in enumerate(conn.execute("SELECT id, task FROM tasks"), start=1):
        task_id, task = row
        listbox_tasks.insert(tk.END, f"{index}: {task}")
    update_task_ids()

# Function to update the task IDs
def update_task_ids():
    global task_ids
    task_ids = [row[0] for row in conn.execute("SELECT id FROM tasks")]

# Create the main window
root = tk.Tk()
root.title("To-Do List")

# Create database or connect to existing one
conn = sqlite3.connect("tasks.db")
conn.execute("CREATE TABLE IF NOT EXISTS tasks (id INTEGER PRIMARY KEY AUTOINCREMENT, task TEXT)")
conn.commit()

# Create GUI elements
frame_tasks = tk.Frame(root)
frame_tasks.pack(pady=10)

listbox_tasks = tk.Listbox(frame_tasks, width=50, height=10, border=0)
listbox_tasks.pack(side=tk.LEFT, fill=tk.BOTH)

scrollbar_tasks = tk.Scrollbar(frame_tasks, orient=tk.VERTICAL, command=listbox_tasks.yview)
scrollbar_tasks.pack(side=tk.RIGHT, fill=tk.Y)

listbox_tasks.config(yscrollcommand=scrollbar_tasks.set)

entry_task = tk.Entry(root, width=50)
entry_task.pack()

button_add_task = tk.Button(root, text="Add Task", width=48, command=add_task)
button_add_task.pack()

button_delete_task = tk.Button(root, text="Delete Task", width=48, command=delete_task)
button_delete_task.pack()

# Initialize the tasks list
update_tasks()

# Run the application
root.mainloop()

# Close the database connection
conn.close()
